In [4]:
!pip install sec_edgar_downloader

In [5]:
import os

import pandas as pd
import requests
from sec_edgar_downloader import Downloader

# Create input and 10k_files subfolders
if not os.path.exists('inputs'):
    os.makedirs('inputs')

In [6]:
# Check if S&P500 list already exists
if os.path.exists('inputs/sp500_list.csv'):
    sp500 = pd.read_csv('inputs/sp500_list.csv')
    tickers = sp500['Symbol'].tolist()
else:
    # Download S&P500 list from Wikipedia page
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    df = pd.read_html(response.content, header=0)[0]
    df.to_csv('inputs/sp500_list.csv', index=False)
    tickers = df['Symbol'].tolist()

In [7]:
import os
import shutil
import tqdm
import concurrent.futures
import re


# Download the last 10-K filed during 2022 for each firm, if not already downloaded
def download_10k(ticker):
    # Set up the downloader
    dl = Downloader()
    # Check if the 10-K for 2022 is already downloaded for this ticker
    pattern = re.compile(rf"10k_files/{ticker}/10-K/\d{{10}}-\d{{2}}-\d{{6}}/full-submission.txt")
    if not any(pattern.match(filename) for filename in os.listdir("10k_files")):
        # Download the 10-K for 2022 for this ticker
        dl.get("10-K", ticker, after="2022-01-01", before="2023-01-01", amount=1)


# Set up a progress bar
pbar = tqdm.tqdm(total=len(tickers))

# Download the 10-Ks for each ticker in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for ticker in tickers:
        futures.append(executor.submit(download_10k, ticker))
    for future in concurrent.futures.as_completed(futures):
        pbar.update(1)

# Close the progress bar
pbar.close()

# Rename the folder sec-edgar-filings to 10k_files
if os.path.exists("sec-edgar-filings"):
    shutil.move("sec-edgar-filings", "10k_files")

100%|██████████| 503/503 [00:00<00:00, 2650.71it/s]


In [1]:
!unzip /content/10k_files.zip
# Rename the folder sec-edgar-filings to 10k_files
if os.path.exists("sec-edgar-filings"):
    shutil.move("sec-edgar-filings", "10k_files")

unzip:  cannot find or open /content/10k_files.zip, /content/10k_files.zip.zip or /content/10k_files.zip.ZIP.


NameError: name 'os' is not defined